In [ ]:
import os
import sys
import warnings
sys.path.append(os.path.dirname(os.getcwd()))
warnings.filterwarnings('ignore')
import pyodbc
from config.configuration import load_config
from backend.db_manager import DBManager

parameters = load_config()

### Table Creation  
After creating the database in the Azure SQL Server service and adding the credentials to the `.env` file, you can run the following cells to create the necessary tables.  

In [4]:
## Create Conversation history table
create_table = False
if create_table:
    sql_query = """
            CREATE TABLE [dbo].[ConversationHistory] (
            [ID]                 INT           IDENTITY (1, 1) NOT NULL,
            [UserID]             INT NOT NULL,
            [DateTime]           DATETIME NOT NULL default(current_timestamp),
            [UserQuery]          NVARCHAR (MAX) NOT NULL,
            [ChatBotAnswer]      NVARCHAR (MAX) NOT NULL,
            PRIMARY KEY CLUSTERED ([ID] ASC)
        );
        """
    conn = pyodbc.connect(parameters['sql_conn_str'])
    cursor = conn.cursor()

    # Table should be created ahead of time in production app.
    cursor.execute(sql_query)

    conn.commit()


In [3]:
## Create users table
create_table = False
if create_table:
    conn = pyodbc.connect(parameters['sql_conn_str'])
    cursor = conn.cursor()

    # Table should be created ahead of time in production app.
    cursor.execute("""
        CREATE TABLE [dbo].[Users] (
        [ID]                 INT           IDENTITY (1, 1) NOT NULL,
        [UserName]           VARCHAR (255) NOT NULL,
        [Email]              VARCHAR (255) NULL,
        [FailedLoginAttemps] INT           CONSTRAINT [DEFAULT_Users_FailedLoginAttemps] DEFAULT ((0)) NOT NULL,
        [LoggedIn]           BIT           CONSTRAINT [DEFAULT_Users_LoggedIn] DEFAULT ((0)) NOT NULL,
        [Name]               VARCHAR (255) NOT NULL,
        [Password]           VARCHAR (255) NOT NULL,
        [Roles]              VARCHAR (255) NOT NULL,
        [max_messages]       INT           CONSTRAINT [DEFAULT_Users_max_messages] DEFAULT ((15)) NULL,
        PRIMARY KEY CLUSTERED ([ID] ASC),
        UNIQUE NONCLUSTERED ([UserName] ASC)
    );
    """)

    conn.commit()

### Testing DBManager funcitons
Once you have populated the tables you can test the different functions from the DBManager class. Below you have some function examples:

In [ ]:
db_manager = DBManager(parameters)
db_manager.get_users_info() 

In [ ]:
user_id = 1 # add the user id
db_manager.get_cant_user_messages(user_id)

In [ ]:
db_manager.validate_user_max_messages(user_id)